In [ ]:
import json
import boto3
import pandas as pd
from io import StringIO
import os

s3 = boto3.client('s3')

def lambda_handler(event, context):
    try:
        # Logs iniciales
        print("Event recibido:", json.dumps(event))
        
        # Nombre del bucket de origen
        bucket_name = "cineposjafet"
        
        # Nombre del bucket de destino
        destination_bucket = "bucketjsondatacine"
        
        # Paginación de objetos en el bucket de origen
        all_objects = []
        continuation_token = None
        
        while True:
            # Listar todos los archivos en el bucket de origen
            if continuation_token:
                response = s3.list_objects_v2(Bucket=bucket_name, ContinuationToken=continuation_token)
            else:
                response = s3.list_objects_v2(Bucket=bucket_name)
            
            if 'Contents' not in response:
                raise Exception(f"No se encontraron archivos en el bucket {bucket_name}")
            
            # Añadir los objetos listados a la lista
            all_objects.extend(response['Contents'])
            
            # Verificar si hay más objetos para listar
            if response.get('IsTruncated'):  # Si hay más objetos
                continuation_token = response.get('NextContinuationToken')
            else:
                break
        
        # Resultado combinado
        all_results = []

        # Procesar cada archivo JSON
        for obj in all_objects:
            file_key = obj['Key']
            print(f"Procesando archivo: {file_key}")
            
            try:
                # Obtener el archivo desde S3
                file_response = s3.get_object(Bucket=bucket_name, Key=file_key)
                json_content = file_response['Body'].read().decode('utf-8')
                
                # Cargar el archivo JSON
                data = json.loads(json_content)
                
                # Convertir el JSON a un DataFrame de pandas
                df = pd.DataFrame(data)
                
                # Procesar el DataFrame con pandas para obtener filas, columnas y NaN
                resultado = {
                    'archivo': file_key,
                    'filas': df.shape[0],
                    'columnas': df.shape[1],
                    'Nan': df.columns[df.isnull().any()].tolist()
                }
                
                print(f"Datos procesados para {file_key}: {resultado}")
                
                # Agregar el resultado a la lista
                all_results.append(resultado)

            except Exception as e:
                print(f"Error procesando el archivo {file_key}: {e}")
                # Continuar con los siguientes archivos aunque haya un error con uno específico
                continue
        
        # Guardar el resultado combinado en el bucket de destino
        directory = "data_cine/"
        combined_file_key = directory + "resultados_combinados.json"
        
        # Subir el archivo combinado, con formato legible (saltos de línea)
        s3.put_object(
            Bucket=destination_bucket,
            Key=combined_file_key,
            Body=json.dumps(all_results, indent=4)  # 'indent=4' agrega saltos de línea y sangría
        )
        
        print(f"Archivo JSON combinado guardado en {combined_file_key}")
        
        return {
            'statusCode': 200,
            'body': json.dumps({"message": "Archivos procesados y resultados guardados", "resultados": all_results})
        }

    except Exception as e:
        print(f"Error durante la ejecución: {e}")
        return {
            'statusCode': 500,
            'body': json.dumps({"error": str(e)})
        }
